**1. Install Necessary Libraries**
We'll need scikit-learn for sparse retrieval and sentence-transformers for dense retrieval.

### **Analogy to Reinforce Understanding**

**Sparse Retrieval**
Analogy: Imagine you have a friend who searches for information in a library by looking for books that contain the exact words you used in your question. If you ask, "What is the color of the sky?" they will find a book that mentions "sky" and "color" exactly as you said it.


**Dense Retrieval**
Analogy: Imagine another friend who understands the meaning behind your question. If you ask, "Describe a lazy dog," they will find a book that talks about a dog being lazy even if the exact words "lazy dog" are not used together. They understand the concept of "lazy" and "dog" and can provide more context.

In [1]:
!pip install scikit-learn sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

**2. Create a Sample Dataset**
We'll create a small dataset with some sample text data.

In [7]:
# Sample dataset
documents = [
    "The sky is blue and beautiful.",
    "Love this blue and beautiful sky!",
    "The quick brown fox jumps over the lazy women.",
    "A king's breakfast has sausages, ham, bacon, eggs, toast, and beans.",
    "I love green eggs, ham, sausages, and bacon!",
    "The brown fox is quick and the blue dog is lazy!",
    "The sky is very blue and the sky is very beautiful today.",
    "The women is lazy but the brown fox is quick!"
]

queries = [
    "What is the color of the sky?",
    "What does the king's breakfast include?",
    "Describe a lazy women."
]


**3. Implement Dense and Sparse Retrievers**
Sparse Retriever (TF-IDF)
Sparse retrieval uses term frequency-inverse document frequency (TF-IDF) to find the most relevant documents.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

def sparse_retriever(query, tfidf_matrix, vectorizer):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_doc_idx = similarities.argsort()[-1]  # Get the index of the most similar document
    return documents[top_doc_idx]


**Dense Retriever (BERT)**
Dense retrieval uses a pre-trained BERT model to encode documents and queries into dense vectors.

In [9]:
from sentence_transformers import SentenceTransformer

# Load pre-trained BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = model.encode(documents)

def dense_retriever(query, document_embeddings, model):
    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity([query_embedding], document_embeddings).flatten()
    top_doc_idx = similarities.argsort()[-1]  # Get the index of the most similar document
    return documents[top_doc_idx]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**4. Compare the Responses**
We can now write a simple function to compare the responses from both retrievers.

In [10]:
def compare_retrievers(queries, tfidf_matrix, vectorizer, document_embeddings, model):
    for query in queries:
        sparse_result = sparse_retriever(query, tfidf_matrix, vectorizer)
        dense_result = dense_retriever(query, document_embeddings, model)
        print(f"Query: {query}")
        print(f"Sparse Retriever Result: {sparse_result}")
        print(f"Dense Retriever Result: {dense_result}")
        print("-" * 50)

# Compare responses
compare_retrievers(queries, tfidf_matrix, vectorizer, document_embeddings, model)


Query: What is the color of the sky?
Sparse Retriever Result: The sky is blue and beautiful.
Dense Retriever Result: The sky is blue and beautiful.
--------------------------------------------------
Query: What does the king's breakfast include?
Sparse Retriever Result: A king's breakfast has sausages, ham, bacon, eggs, toast, and beans.
Dense Retriever Result: A king's breakfast has sausages, ham, bacon, eggs, toast, and beans.
--------------------------------------------------
Query: Describe a lazy women.
Sparse Retriever Result: The quick brown fox jumps over the lazy women.
Dense Retriever Result: The women is lazy but the brown fox is quick!
--------------------------------------------------


**Query: "What is the color of the sky?"**
Sparse Retriever Result: "The sky is blue and beautiful."
Dense Retriever Result: "The sky is blue and beautiful."
Explanation: Both methods returned the same document because it contains the exact terms "sky" and "blue," which are key to answering the query. This example shows that both methods can perform equally well when the query terms match the document terms directly.

**Query: "Describe a lazy dog."**
Sparse Retriever Result: "The quick brown fox jumps over the lazy dog."
Dense Retriever Result: "The dog is lazy but the brown fox is quick."
Explanation: Here, we see a difference in the results:
Sparse Retrieval: This method returned a document that contains the exact phrase "lazy dog," even though the context is not fully aligned with describing a lazy dog.
Dense Retrieval: This method understood the query's context better, providing a document that not only mentions a lazy dog but also describes it as "lazy but the brown fox is quick," which gives more context about the dog's behavior.